In [1]:
include("../src/ECM_TDA.jl")
using .ECM_TDA
using DataFrames
using Ripserer
using NPZ
using PersistenceDiagrams
using JLD2
using FileIO
using PersistenceDiagrams
using DelimitedFiles
using Distributions
using Statistics
using Measures
using TiffImages
using CSV
using DataFrames
using Plots
using Images
using StatsBase

WebIO._IJuliaInit()

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/Users/irisyoon/.julia/packages/Plots/yJrrq/src/backends/hdf5.jl", 36, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00000001048432cf, Ptr{Nothing} @0x00000001048d8206, Ptr{Nothing} @0x00000001048d923b, Ptr{Nothing} @0x00000001048d7b94, Ptr{Nothing} @0x00000001048d7f0c, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x00000001048f2844, Ptr{Nothing} @0x00000001048f1f65, Ptr{Nothing} @0x00000001048f2604, Ptr{Nothing} @0x00000001048f2604, Ptr{Nothing} @0x00000001048f3134, Ptr{Nothing} @0x0000000117b8b4e7, Ptr{Nothing} @0x00000001048c1fbf, Ptr{Nothing} @0x0000000117bdc312, Ptr{Nothing} @0x0000000117812a65, Ptr{Nothing} @0x00000001048c1fbf, Ptr{Nothing} @0x00000001048ce839, Ptr{Nothing} @0x0000000105e12a8e, Ptr{Nothing} @0x00000001048c1fbf, Ptr{Nothing} @0x00000001048d925f, Ptr{Nothing} @0x00000001048d7b94, Ptr{Nothing} @0x000000

In [2]:
c = "#249EA0" # teal

"#249EA0"

In [4]:
function sample_points_from_ECM_directory(ECM_directory, min_sample, max_sample, low, high, c_ECM;
                                         invert = true,
                                         sample_plot_directory = "",
                                         sample_CSV_directory = "")
    # create directories for sample plot and CSV if it doesn't exist
    if sample_plot_directory != "" 
        ispath(sample_plot_directory) || mkpath(sample_plot_directory)
    end
    if sample_CSV_directory != ""
        ispath(sample_CSV_directory) || mkpath(sample_CSV_directory)
    end


    files = [item for item in walkdir(ECM_directory)][1][3]
    for filename in files
        if filename[1] != '.'
            img = Array(Images.load(ECM_directory * filename))
            figure_file = split(filename,".")[1] * ".pdf"

            # compute (inverted) mean pixel value of image
            #img_mean_inv = 1- mean(Float64.(img))
            img_mean_inv = mean(Float64.(img))

            # compute number of points to sample
            n_sample = Int64(round(ECM_TDA.n_ECM_samples(img_mean_inv, low, high, min_sample, max_sample)))

            # sample points
            resampled, points, sampled, island_idx = sample_ECM_points(img, 
                                                                        vicinity = 100, 
                                                                        n_points = 5, 
                                                                        n_samples = n_sample,
                                                                        invert = invert)

            # plot the results 
            p = scatter(resampled[:,1], resampled[:,2], yflip = :true, c = c_ECM, label = "", frame = :box, ticks = [], 
                        aspect_ratio = :equal,
                        xlims = (0, 4000),
                        ylims = (0, 4000),
                        size = (500, 500))
            savefig(sample_plot_directory * figure_file)

            # save sampled points to CSV
            csv_file = split(filename, ".")[1] * ".csv"
            df = DataFrame(resampled, [:x, :y])
            CSV.write(sample_CSV_directory * csv_file, df)
        end
    end
end

function sample_collagen_points(image; 
    scale = 1,
    vicinity = 50,
    n_points = 5,
    n_samples = 2000)
    
    image = image ./ scale
    
    ### 1. Sample points according to binomial distribution
    # for each pixel location with pixel value p, sample the point according to Binomial(1, p^2)
    sampled_img = image.^2 .> rand(Uniform(0,1), size(image))
    
    # get index of sampled points
    inds = Tuple.(findall(!iszero, sampled_img))
    points = hcat(first.(inds), last.(inds))
    
    ### 2. Remove island points 
    image_size = size(image,1)
    island_idx = ECM_TDA.find_island_idx(points, sampled_img, image_size; vicinity = vicinity, n_points = n_points) 
    
    ### 3. Downsample
    include_idx = setdiff(1:size(points,1), island_idx)

    if length(include_idx) <= n_samples
        sampled_idx = include_idx
    else 
        sampled_idx = sample(include_idx, n_samples, replace = false)
    end
    sampled_points = points[sampled_idx,:]

    # for consistency
    points = points[:,[2,1]] # needed for consistency when plotted 
    sampled_points = sampled_points[:,[2,1]] # needed for consistency when plotted 
    
    return sampled_points, points, sampled_img, island_idx
end

function sample_points_from_ECM_directory(ECM_directory, min_sample, max_sample, low, high, c_ECM;
                                         invert = true,
                                         sample_plot_directory = "",
                                         sample_CSV_directory = "")
    # create directories for sample plot and CSV if it doesn't exist
    if sample_plot_directory != "" 
        ispath(sample_plot_directory) || mkpath(sample_plot_directory)
    end
    if sample_CSV_directory != ""
        ispath(sample_CSV_directory) || mkpath(sample_CSV_directory)
    end


    files = [item for item in walkdir(ECM_directory)][1][3]
    for filename in files
        if filename[1] != '.'
            img = Array(Images.load(ECM_directory * filename))
            figure_file = split(filename,".")[1] * ".pdf"

            # compute (inverted) mean pixel value of image
            #img_mean_inv = 1- mean(Float64.(img))
            img_mean_inv = mean(Float64.(img))

            # compute number of points to sample
            n_sample = Int64(round(ECM_TDA.n_ECM_samples(img_mean_inv, low, high, min_sample, max_sample)))

            # sample points
            resampled, points, sampled, island_idx = sample_ECM_points(img, 
                                                                        vicinity = 100, 
                                                                        n_points = 5, 
                                                                        n_samples = n_sample,
                                                                        invert = invert)

            # plot the results 
            p = scatter(resampled[:,1], resampled[:,2], yflip = :true, c = c_ECM, label = "", frame = :box, ticks = [], 
                        aspect_ratio = :equal,
                        xlims = (0, 4000),
                        ylims = (0, 4000),
                        size = (500, 500))
            savefig(sample_plot_directory * figure_file)

            # save sampled points to CSV
            csv_file = split(filename, ".")[1] * ".csv"
            df = DataFrame(resampled, [:x, :y])
            CSV.write(sample_CSV_directory * csv_file, df)
        end
    end
end

function get_ECM_mean_pixels(directory)
    files =  [item for item in walkdir(directory)][1][3]
    files = [f for f in files if f[1] != '.']
    ECM_mean_pixels = []
    for filename in files
        img = Array(Images.load(directory * filename))
        push!(ECM_mean_pixels, mean(Float64.(img)))
    end
    return ECM_mean_pixels
end

function get_row_col_lim(dir_2000)
    files =  [item for item in walkdir(dir_2000)][1][3]
    files = [x for x in files if (x[end-3:end] == ".tif") & (x[1] != '.')]

    rows = [parse(Int32, split(f, "_")[3]) for f in files]
    cols = [parse(Int32, split(f,"_")[4]) for f in files]

    max_row = maximum(rows)
    max_col = maximum(cols)

    if iseven(max_row)
        row_lim = max_row - 2
    else
        row_lim = max_row
    end

    if iseven(max_col)
        col_lim = max_col - 2
    else
        col_lim = max_col
    end
    return row_lim, col_lim

end

function stitch_2000_tiles_to_4000(dir_2000, dir_4000, row_lim, col_lim)
    for i =0:2:row_lim
        for j = 0:2:col_lim
            
            input1 = dir_2000 * "image_tile_" * lpad(i, 5, "0") * "_" * lpad(j, 5, "0") * "_psr.tif"
            input2 = dir_2000 * "image_tile_" * lpad(i, 5, "0") *  "_" * lpad(j+1, 5, "0") * "_psr.tif"
            input3 = dir_2000 * "image_tile_" * lpad(i+1, 5, "0") * "_" * lpad(j, 5, "0") * "_psr.tif"
            input4 = dir_2000 * "image_tile_" * lpad(i+1, 5, "0") * "_" * lpad(j+1, 5, "0") * "_psr.tif"
            
            new_i = lpad(Int32(i/2), 5, "0")
            new_j = lpad(Int32(j/2), 5, "0")
    
            outputfile = dir_4000 * new_i * "_" * new_j * "_psr.tif"
            mycommand = `magick montage -tile 2x2 -geometry +0+0 $input1 $input2 $input3 $input4 $outputfile`
            run(mycommand)
        end
    end
end

function plot_wholeslide_points(LTX)
    gr()
    # plot the (stitched) points
    p = plot()

    sample_CSV_directory = "data_TDA/LTX" * LTX * "/points_CSV/"
    for f in [item for item in walkdir(sample_CSV_directory)][1][3]
        s = split(f, ".")[1]
        row, col = split(s, "_")
        row = parse(Int32, row)
        col = parse(Int32, col)

        row_adjust = row * 4000
        col_adjust = col * 4000
        # load points
        df = CSV.read(sample_CSV_directory * s * ".csv")
        scatter!(p, df[:,:x] .+ col_adjust, df[:,:y] .+ row_adjust,
                yflip = :true,
                label = "", markersize = 1, markerstrokewidth = 0,
                c = c)
    end

    plot(p, aspect_ratio = :equal, size = (3000, 3000))
    savefig("data_TDA/LTX" * LTX * "/sampled_points_whole.png")
end

sample_points_from_ECM_directory (generic function with 1 method)

In [ ]:
# specify LTXs. These were selected by Erik for their diversity 
#LTXs = ["001", "013", "021", "034", "041", "050", "051", "073", "079", "092",
# "100", "108", "115", "142", "143", "145", "185", "206", "210", "221" ]


# Sample points, compute PD
*  stitch 4 (2000x2000) tiles to create tiles of size 4000 x 4000
* sample points
    - To determine the number of points to sample, use the same function from the original data (dataset 1), since all images come from the same imaging modality.
* `low`, `high`, `min_sample`, `max_sample`: parameters from `../2_sample_points_from_ECM.ipynb`

* `row_num`, `col_num` are the maximum indices of the 2000 x 2000 tiles. 
* Value depend on LTX

In [11]:
low = 0.0006376811274509953
high = 0.4025769968137255
min_sample = 100 
max_sample = 5000

5000

In [15]:
for LTX in LTXs
    println(LTX)
    dir_2000 = "/Volumes/My Passport/iris_data/LTX" * LTX * "/tile_size_2000/whole_slide/PSR/deconvolutions/ScanRegion0/psr/inverted_grayscale/"
    dir_4000 = "/Volumes/My Passport/iris_data/LTX" * LTX * "/tile_size_4000/"
    
    ### create stitched tiles ###
    # create paths
    ispath(dir_4000) || mkpath(dir_4000)

    #stitch 2000 x 2000 tiles to create tiles of size 4000 x 4000
    row_lim, col_lim = get_row_col_lim(dir_2000)
    stitch_2000_tiles_to_4000(dir_2000, dir_4000, row_lim, col_lim)

    ### sample points ###
    sample_plot_directory = "data_TDA/LTX" * LTX * "/sampled_points/"
    sample_CSV_directory = "data_TDA/LTX" * LTX * "/points_CSV/"

    # create paths
    ispath(sample_plot_directory) || mkpath(sample_plot_directory)
    ispath(sample_CSV_directory) || mkpath(sample_CSV_directory)

    sample_points_from_ECM_directory(dir_4000, min_sample, max_sample, low, high, c;
                                    invert = false,
                                    sample_plot_directory = sample_plot_directory,
                                    sample_CSV_directory = sample_CSV_directory)


    # stitch together the sampled points
    plot_wholeslide_points(LTX)

    ##### Compute PD #####
    sample_CSV_directory = "data_TDA/LTX" * LTX * "/points_CSV/"
    PD0_dir = "data_TDA/LTX" * LTX * "/PD0/"
    PD1_dir = "data_TDA/LTX" * LTX * "/PD1/"

    # create paths
    ispath(PD0_dir) || mkpath(PD0_dir)
    ispath(PD1_dir) || mkpath(PD1_dir)

    # compute PD 
    PD0_ECM = Dict()
    PD1_ECM = Dict()
    for f in [item for item in walkdir(sample_CSV_directory)][1][3]
        name = split(f, ".")[1]
        df = CSV.read(sample_CSV_directory * f)

        if size(df,1) != 0
            PD0, PD1 = run_PH(df)

            # save
            writedlm(PD0_dir * name * ".csv", PD0, ",")
            writedlm(PD1_dir * name * ".csv", PD1, ",")

            # combine to dictionary
            PD0_ECM[name] = PD0
            PD1_ECM[name] = PD1
        else
            PD0 = nothing
            PD1 = nothing

            writedlm(PD0_dir * name * ".csv", zeros(), ",")
            writedlm(PD1_dir * name * ".csv", zeros(), ",")
            # combine to dictionary
            PD0_ECM[name] = PD0
            PD1_ECM[name] = PD1
        end
    end

    # save PD
    save("data_TDA/LTX" * LTX * "/PD.jld2", "PD0", PD0_ECM, "PD1", PD1_ECM)

    ### compute PI  
    # convert array to Ripserer PD
    PH0 = Dict(k => ECM_TDA.array_to_ripsererPD(v) for (k,v) in PD0_ECM if v != nothing)
    PH1 = Dict(k => ECM_TDA.array_to_ripsererPD(v) for (k,v) in PD1_ECM if v != nothing)

    # compute PI
    PI0 = ECM_TDA.compute_PI(PH0);
    PI1 = ECM_TDA.compute_PI(PH1);
    # save PI
    save("data_TDA/LTX" * LTX * "/PI.jld2","PI0", PI0, "PI1", PI1)

end

143
145
